In [205]:
from bs4 import BeautifulSoup
import requests

In [206]:
#Table A - GDP Table

url = "https://en.wikipedia.org/wiki/List_of_cities_by_GDP#Cities_by_wealth"
soup = BeautifulSoup(requests.get(url).content, "html.parser")

In [207]:
A_table = soup.find_all("table")[2]
A_table_2 = soup.find_all("table")[3]

In [208]:
A_headers = A_table.find_all("th")
A_headers = [title.text.strip() for title in A_headers]


A_headers[0] = "City Proper/Metropolitan Area"
A_headers[1] = "Country/Region"
A_headers[-1] = "Metropolitan Population"
A_headers[2] = 'Official est. GDP up to date (billion US$)'
A_headers.insert(0, "GDP ID")
A_headers.insert(4, "GDP Year")
A_headers.insert(6, "Population Year")
print(A_headers)

['GDP ID', 'City Proper/Metropolitan Area', 'Country/Region', 'Official est. GDP up to date (billion US$)', 'GDP Year', 'Metropolitan Population', 'Population Year']


In [209]:
import pandas as pd


In [210]:
def remove_until_parenthesis(s):
    for i in range(len(s) - 1, -1, -1):
        if s[i] == ')':
            return s[:i+1]
    return s


In [211]:
TABLE_A = pd.DataFrame(columns=A_headers)
A_rows = A_table.find_all("tr")
count = 0
for row in A_rows:

    row_data = row.find_all('td')
    indiv_row_data = [data.text.strip() for data in row_data]
    #First row is [] so when count >0
    if count>0:
        # Extract data that are in same cells on web page
        GDP_split = indiv_row_data[2].split()
        indiv_row_data.pop(2)
        indiv_row_data.insert(2, GDP_split[0])
        try:
            GDP_split_year = remove_until_parenthesis(GDP_split[1])
        except IndexError:
            GDP_split_year = ""
        GDP_split_year = GDP_split_year.replace("(", "").replace(")", "")
        indiv_row_data.insert(3, GDP_split_year)
        POP_split = indiv_row_data[4].split()
        indiv_row_data.pop(4)
        indiv_row_data.insert(4, POP_split[0])
        try:
            POP_split_year = remove_until_parenthesis(POP_split[1])
        except IndexError:
            POP_split_year = ""
        POP_split_year = POP_split_year.replace("(", "").replace(")", "")
        indiv_row_data.insert(5, POP_split_year)
        indiv_row_data.insert(0, count)

        #Formatting numbers to be numbers GDP: Float. Pop, GDP year, pop year: int
        #adding "" to cells that contain comma
        for i in range(1, len(indiv_row_data)):
            if i>2:
                if i ==3:
                    indiv_row_data[i] = float(indiv_row_data[i].replace(",", ""))
                else:
                    try:
                        indiv_row_data[i] = int(indiv_row_data[i].replace(",", ""))
                    except ValueError: #two years in one Ex: 2014-15
                        #Sol: Let it be the later year (ex 2015)
                        if indiv_row_data[i] == "": next
                        else: indiv_row_data[i] = int(indiv_row_data[i][:-3])+1
            else:
                if indiv_row_data[i].find(",") != -1:
                    indiv_row_data[i] = '"' + indiv_row_data[i] + '"'

        print(indiv_row_data)

        #inserting into df
        length = len(TABLE_A)
        TABLE_A.loc[length] = indiv_row_data
    count+=1


A2_rows = A_table_2.find_all("tr")
count1 = 0
for row in A2_rows:
    row_data = row.find_all("td")
    indiv_row_data = [data.text.strip() for data in row_data]
    if count1>0:
        indiv_row_data = indiv_row_data[:3]
        #Rearranging data to fit columns of table A
        placeholder = indiv_row_data[1]
        indiv_row_data[1] = indiv_row_data[2]
        indiv_row_data[2] = placeholder
        #Filling blank space in data with NA
        indiv_row_data.append("")
        indiv_row_data.append("")
        indiv_row_data.append("")

        indiv_row_data.insert(0, count+count1-1)

        #Formatting numbers to be numbers GDP: Float. Pop, GDP year, pop year: int
        #adding "" to cells that contain comma
        for i in range(1, 4):
                if i ==3:
                        indiv_row_data[i] = float(indiv_row_data[i].replace(",", ""))
                else:
                    if indiv_row_data[i].find(",") != -1:
                        indiv_row_data[i] = '"' + indiv_row_data[i] + '"'

        print(indiv_row_data)
        #Insert into table 
        length = len(TABLE_A)
        TABLE_A.loc[length] = indiv_row_data
    count1 +=1

[1, 'A Coruña metropolitan area', 'Spain', 28.819, 2020, 1121815, 2020]
[2, 'Aachen', 'Germany', 24.296, 2020, 249070, 2021]
[3, 'Aalborg', 'Denmark', 31.855, 2021, 219487, 2021]
[4, 'Aarhus', 'Denmark', 54.927, 2021, 352751, 2021]
[5, '"Abbotsford, British Columbia"', 'Canada', 7.435, 2021, 205834, 2021]
[6, '"Aberdeen, Scotland"', 'United Kingdom', 23.0, 2020, 489840, 2020]
[7, 'Abidjan', 'Ivory Coast', 27.0, 2017, 5600000, 2020]
[8, '"Abilene, TX MSA"', 'United States', 10.306623, 2023, 181591, 2023]
[9, 'Abu Dhabi metropolitan area', 'United Arab Emirates', 277.93, 2023, 1740000, 2023]
[10, 'Addis Ababa', 'Ethiopia', 61.2, 2023, 5400000, 2023]
[11, 'Adelaide', 'Australia', 64.461, 2019, 1344368, 2019]
[12, 'Ahmedabad', 'India', 136.1, 2023, 9300000, 2023]
[13, 'Aix-Marseille-Provence Metropolis', 'France', 135.038, 2021, 1680000, 2021]
[14, '"Akron, OH MSA"', 'United States', 47.285487, 2023, 698398, 2023]
[15, '"Albany, GA MSA"', 'United States', 7.782093, 2023, 145508, 2023]
[16,

In [217]:
TABLE_A

,GDP ID,City Proper/Metropolitan Area,Country/Region,Official est. GDP up to date (billion US$),GDP Year,Metropolitan Population,Population Year
0,1,A Coruña metropolitan area,Spain,28.819000,2020,1121815,2020
1,2,Aachen,Germany,24.296000,2020,249070,2021
2,3,Aalborg,Denmark,31.855000,2021,219487,2021
3,4,Aarhus,Denmark,54.927000,2021,352751,2021
4,5,"""Abbotsford, British Columbia""",Canada,7.435000,2021,205834,2021
...,...,...,...,...,...,...,...
994,995,Delhi,India,521.500000,,,
995,996,Rhine-Ruhr,Germany,634.437000,,,
996,997,Milan,Italy,233.596000,,,
997,998,New York,United States,2298.868394,,,


In [221]:
TABLE_A.to_csv("tableA.csv", index=False)